# Sigmoid Regression

We use the `scipy.optimize.curve_fit` function for the estimation through non-linear regression of the parameters of the model described in equation 25. 

In [ ]:
import numpy as np
import scipy.optimize

import dotdot
from neuromodel import Model, Offers, run_model
import neuromodel.graphs

%matplotlib notebook

In [ ]:
ΔA, ΔB, n = (0, 20), (0, 20), 4000
offers = Offers(ΔA=ΔA, ΔB=ΔB, n=n, random_seed=1)

x_offers = ((1, 0), (20, 1), (16, 1), (12, 1), (8, 1), (4, 1), # specific offers for Figures 4C, 4G, 4K
            (1, 4), (1, 8), (1, 12), (1, 16), (1, 20), (0, 1))

In [ ]:
def compute_fig4_data():
    model = Model(n=n, ΔA=ΔA, ΔB=ΔB, random_seed=0)

    filename='data/fig4[{}].pickle'.format(n)
    return run_model(model, offers, history_keys=('r_ovb', 'r_2', 'r_I'), filename=filename)

In [ ]:
analysis = compute_fig4_data()

In [ ]:
def func(x, a_0, a_1, a_2, a_3, a_4, a_5):
    x_a, x_b = x
    X = a_0 + a_1 * x_a + a_2 * x_b + a_3 * x_a*x_a + a_4 * x_b*x_b + a_5 * x_a*x_b
    return 1/(1 + np.exp(-X))

In [ ]:
X_A, X_B, choice_B = [], [], []
for (x_a, x_b), (n_a, n_b) in sorted(analysis.choices.items()):
    if x_a != 0 or x_b != 0:
        X_A.append(x_a)
        X_B.append(x_b)
        choice_B.append(n_b / (n_a + n_b))

In [ ]:
a_opt, a_cov = scipy.optimize.curve_fit(func, [X_A, X_B], choice_B, bounds=((-20,)*6, (20,)*6))

In [ ]:
print('Regression results')
for i, a in enumerate(a_opt):
    print('a_{} = {: .4f}'.format(i, a))

In [ ]:
# computing the regressed model over all possible quantities by 0.5 increments.
X_A_reg = np.arange(0, 20.5, 0.5)
X_B_reg = np.arange(0, 20.5, 0.5)
X_A_reg, X_B_reg = np.meshgrid(X_A_reg, X_B_reg)
choice_B_reg = 100*func((X_A_reg, X_B_reg), *a_opt)

In [ ]:
neuromodel.graphs.regression(X_A, X_B, 100*np.array(choice_B), X_A_reg, X_B_reg, choice_B_reg)